# Anomaly Detection: EfficientAD model, Mvtec dataset

In [ ]:
%pip install anomalib

In [ ]:
pip install lightning

In [ ]:
pip install kornia

In [ ]:
!pip install --upgrade anomalib

In [ ]:
import anomalib.data as ad

# anomalib.data modülü içinde tanımlı olan tüm alt sınıfları, fonksiyonları, objeleri ve alt modülleri listeler.
print(dir(ad))

- ImageNet veri seti, Anomalib içinde hazır bir DataModule veya Dataset sınıfı olarak yer almıyor.

- Yani MVTecAD Dataset gibi doğrudan çağırabileceğin bir ImageNetDataset tanımı yok.

- Çünkü ImageNet genelde sınıflandırma problemleri için kullanılır; anomalib ise anomali tespiti problemlerine odaklanır.

In [ ]:
from pathlib import Path

# NOTE: Provide the path to the dataset root directory.
#   If the datasets is not downloaded, it will be downloaded
#   to this directory.
dataset_root = Path.cwd().parent.parent.parent / "datasets" / "MVTecAD"

In [ ]:
# import shutil

# # 2. Check if the directory exists and delete it if it does
# if dataset_root.exists() and dataset_root.is_dir():
#     shutil.rmtree(dataset_root)
#     print(f"Deleted existing dataset directory: {dataset_root}")
# else:
#     print(f"Dataset directory not found: {dataset_root}")

In [ ]:
import numpy as np
from PIL import Image
from torchvision.transforms.v2.functional import to_pil_image

from anomalib.data import MVTecAD, MVTecADDataset

In [ ]:
datamodule = MVTecAD(
    root=dataset_root,
    category="bottle",
    train_batch_size=1, # 32 hata verdi
    eval_batch_size=32,
    num_workers=0,
)

In [ ]:
datamodule.prepare_data()
datamodule.setup()

In [ ]:
# Train images
i, data = next(enumerate(datamodule.train_dataloader()))
print(data.image.shape)

In [ ]:
# Test images
i, data = next(enumerate(datamodule.test_dataloader()))
print(data.image.shape, data.gt_mask.shape)

In [ ]:
img = to_pil_image(data.image[1].clone())
msk = to_pil_image(data.gt_mask[1].int() * 255).convert("RGB")

Image.fromarray(np.hstack((np.array(img), np.array(msk))))

In [ ]:
# MVTec dataset
mvtec_dataset_train = MVTecADDataset(
    root=dataset_root,
    category="bottle",
    split="train",
)
print(len(mvtec_dataset_train))
sample = mvtec_dataset_train[1]
print(sample.image.shape, sample.image_path, sample.gt_label)

In [ ]:
# MVTec Classification Test Set
mvtec_dataset_test = MVTecADDataset(
    root=dataset_root,
    category="bottle",
    split="test",
)
print(len(mvtec_dataset_test))
sample = mvtec_dataset_test[1]
print(sample.image.shape, sample.image_path, sample.gt_label)

In [ ]:
img = to_pil_image(sample.image.clone())
msk = to_pil_image(sample.gt_mask.int() * 255).convert("RGB")

Image.fromarray(np.hstack((np.array(img), np.array(msk))))

In [ ]:
!pip install openvino-dev

In [ ]:
pip install FrEIA

In [ ]:
pip install python-dotenv

In [ ]:
pip install open_clip_torch

In [ ]:
!pip install anomalib timm --quiet

In [ ]:
import anomalib.models as ef

# modülü içinde tanımlı olan tüm alt sınıfları, fonksiyonları, objeleri ve alt modülleri listeler.
print(dir(ef))

In [ ]:
from anomalib.models import EfficientAd
from anomalib.data import MVTec
from anomalib.engine import Engine
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

import numpy as np
from PIL import Image
from pathlib import Path

In [ ]:
!pip install --upgrade anomalib

In [ ]:
!pip install --upgrade --force-reinstall anomalib

In [ ]:
model = EfficientAd()

In [ ]:
# Eğitim sırasında kullanılacak callback'ler
callbacks = [
    ModelCheckpoint(
        # pixel_AUROC yerine train_loss veya train_loss_epoch gibi bir metrik kullanın
        monitor="train_loss_epoch",
        mode="min"  # train_loss için mode genellikle "min" olur
    ),
    EarlyStopping(
        # pixel_AUROC yerine train_loss veya train_loss_epoch gibi bir metrik kullanın
        monitor="train_loss_epoch",
        mode="min",  # train_loss için mode genellikle "min" olur
        patience=3
    ),
]

In [ ]:
# Eğitim motoru
engine = Engine(
    callbacks=callbacks,
    accelerator="auto",
    devices=1,
    logger=False
)


In [ ]:
# Trains the model using the dataset
engine.fit(datamodule=datamodule, model=model)

In [ ]:
print("Eğitimin maksimum epoch sınırı:", engine.trainer.max_epochs)


In [ ]:
# Eğitim sürecinde maksimum kaç epoch çalıştırılacağı
print("🔢 Maksimum Epoch Sayısı (max_epochs):", engine.trainer.max_epochs)

# Eğitim sürecinde en az kaç epoch çalıştırılacağı
print("🔢 Minimum Epoch Sayısı (min_epochs):", engine.trainer.min_epochs)

# Şu anda eğitim hangi epoch'ta? (0'dan başlar)
print("📍 Şu Anki Epoch (current_epoch):", engine.trainer.current_epoch)

# Eğitimde hangi hızlandırıcı (GPU, CPU, MPS) kullanılıyor?
print("⚡ Kullanılan Hızlandırıcı (accelerator):", engine.trainer.accelerator.__class__.__name__)

# Kullanılan cihaz sayısı (örneğin 1 GPU veya CPU)
print("🖥️ Cihaz Sayısı (devices):", engine.trainer.num_devices)

# Validation (doğrulama) işlemi her ne sıklıkla yapılacak?
# 1.0: Her epoch sonunda, 0.5: Yarım epoch'ta bir
print("🔁 Validation Sıklığı (val_check_interval):", engine.trainer.val_check_interval)

# Eğitim verisinin ne kadarı kullanılacak?
# 1.0: %100'ü, 0.5: %50’si
print("📦 Eğitim Verisi Kullanım Oranı (limit_train_batches):", engine.trainer.limit_train_batches)

# Validation verisinin ne kadarı kullanılacak?
# 1.0: %100'ü, 0.2: %20’si
print("📦 Validation Verisi Kullanım Oranı (limit_val_batches):", engine.trainer.limit_val_batches)

# Kaç adımda bir log (çıktı) yazılsın?
print("📝 Loglama Aralığı (log_every_n_steps):", engine.trainer.log_every_n_steps)


In [ ]:
engine.test(datamodule=datamodule, model=model)

In [ ]:
data_path = dataset_root / "bottle/test/broken_large/001.png"
predictions = engine.predict(model=model, data_path=data_path)
prediction = predictions[0]  # Get the first and only prediction

In [ ]:
print(
    f"Image Shape: {prediction.image.shape},\n"
    f"Anomaly Map Shape: {prediction.anomaly_map.shape}, \n"
    f"Predicted Mask Shape: {prediction.pred_mask.shape}",
)

In [ ]:
image_path = prediction.image_path[0]
image_size = prediction.image.shape[-2:]
image = np.array(Image.open(image_path).resize(image_size))

In [ ]:
pred_score = prediction.pred_score[0]
pred_labels = prediction.pred_label[0]
print(pred_score, pred_labels)

In [ ]:
from anomalib.deploy import ExportType, OpenVINOInferencer

In [ ]:
pip install onnx

In [ ]:
engine.export(
    model=model,
    export_type=ExportType.OPENVINO,
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from anomalib.data.utils import read_image

image_path = prediction.image_path[0]
image = read_image(path=image_path)
plt.imshow(image)

In [ ]:
# Checking the Model Save Path
output_path = Path(engine.trainer.default_root_dir)
print(output_path)

In [ ]:
# Verifying the Existence of the OpenVINO Model
openvino_model_path = output_path / "weights" / "openvino" / "model.bin"
print(openvino_model_path.exists())


In [ ]:
import time


# Zamanlamayı başlat
start_time = time.time()

# Loading the OpenVINO Model and runs it on an Intel processor
inferencer = OpenVINOInferencer(
    path=openvino_model_path,  # Path to the OpenVINO IR model.
    device="CPU",  # We would like to run it on an Intel CPU.
)

# Zamanı ölç
model_loading_time = time.time() - start_time
print(f"Model loading time: {model_loading_time:.2f} seconds")

In [ ]:
from anomalib.utils.visualization import ImageResult

# Performing Inference (çıkarım yapma) on an Image Using the Model
start_time = time.time()
predictions = inferencer.predict(image=image_path)

# Zamanı ölç
inference_time = time.time() - start_time
print(f"Inference time: {inference_time:.2f} seconds")

# 3. Çıktıları görselleştirme
start_time = time.time()

# Converting the Output for Visualization
predictions = ImageResult.from_dataset_item(predictions.items[0])  # convert to imageresult for visualization


# Zamanı ölç
visualization_time = time.time() - start_time
print(f"Visualization time: {visualization_time:.2f} seconds")

# Toplam süre
total_time = model_loading_time + inference_time + visualization_time
print(f"Total time: {total_time:.2f} seconds")

In [ ]:
# Printing the Model Output
print(predictions.pred_score, predictions.pred_label)

In [ ]:
# Görselleri aynı figürde yan yana göstermek için subplot kullanıyoruz.
fig, axes = plt.subplots(1, 5, figsize=(20, 5))

# Orijinal görüntü
axes[0].imshow(predictions.image)
axes[0].set_title("Original Image")

# Modelin tahmin ettiği ham anomali haritası
axes[1].imshow(predictions.anomaly_map)
axes[1].set_title("Anomaly Map")

# Isı haritası (Ham anomali haritası, orijinal görüntüyle birleştirilmiş hali)
axes[2].imshow(predictions.heat_map)
axes[2].set_title("Heat Map")

# Segmentasyon maskesi (Modelin belirlediği anomali bölgeleri)
axes[3].imshow(predictions.pred_mask)
axes[3].set_title("Predicted Mask")

# Segmentasyon maskesi ile orijinal görüntüyü birleştirilmiş hali
axes[4].imshow(predictions.segmentations)
axes[4].set_title("Segmentations")

# Eksensiz gösterim
for ax in axes:
    ax.axis("off")

# Görselleri göster
plt.tight_layout()
plt.show()
